<a href="https://colab.research.google.com/github/mayankpahade611/Rag-pdf-assistant/blob/main/RAG_Based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers chromadb transformers pdfplumber pypdf langchain-community accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 136.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.3 MB/s eta 0:0

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
import pdfplumber
from transformers import pipeline

In [ ]:
!pip install pdfplumber pypdf2 unidecode regex
!pip install pytesseract pillow
!apt-get update -qq && apt-get install -y -qq tesseract-ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 18.9 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
import os
import io
import json
import re
import pdfplumber
from typing import List, Dict
from unidecode import unidecode

In [ ]:
page_num_regex = re.compile(r'^\s*(page|pg|p)\.?\s*\d+\s*$', re.I)
trailing_page_num = re.compile(r'\s*\b\d{1,3}\s*$')  # common page numbers at line end

def normalize_whitespace(text: str) -> str:
    # Basic whitespace and unicode normalization
    text = unidecode(text)                  # normalize unicode characters
    text = text.replace('\r', '\n')         # unify line endings
    text = re.sub(r'\n{3,}', '\n\n', text)  # collapse many newlines
    text = re.sub(r'[ \t]+', ' ', text)     # collapse spaces/tabs
    text = text.strip()
    return text

In [ ]:
def remove_header_footer_lines(page_lines: List[str], context_signature=None) -> List[str]:
    """
    Heuristic: if a line repeats across >50% of pages (same exact text),
    consider it a header/footer and drop it. Optionally pass a context_signature
    (like filename) to exclude lines that are genuinely unique.
    """
    # This function will be used by a wrapper that has visibility across pages.
    # Keep placeholder for clarity — actual removal is done later in the multi-page flow.
    return page_lines

In [ ]:
def clean_page_text(text: str) -> str:
    lines = [ln.strip() for ln in text.splitlines()]
    cleaned_lines = []
    for ln in lines:
        if not ln:
            continue
        # drop obvious page numbers or small footer-like lines
        if page_num_regex.match(ln):
            continue
        # drop isolated line with just a small integer (common page number)
        if len(ln) < 6 and ln.strip().isdigit():
            continue
        # drop lines that are just hyphenation markers
        if ln.strip() in ['-', '—', '–']:
            continue
        # remove trailing page numbers that appear at line end
        ln = trailing_page_num.sub('', ln)
        cleaned_lines.append(ln)
    text = '\n'.join(cleaned_lines)
    text = normalize_whitespace(text)
    # de-hyphenate word-breaks like "exam-\nple" -> "example"
    text = re.sub(r'([a-zA-Z])-\\?\n([a-zA-Z])', r'\1\2', text)
    return text

In [ ]:
def extract_text_from_pdf(path: str, detect_headers_footers: bool = True) -> List[Dict]:
    """
    Extracts text from each page of a PDF and returns list of dicts:
      {'doc_id': <filename>, 'page_num': i, 'raw_text': ..., 'clean_text': ...}
    """
    doc_id = os.path.basename(path)
    pages = []
    with pdfplumber.open(path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            try:
                raw = page.extract_text() or ""
            except Exception as e:
                # fallback: empty string
                raw = ""
            pages.append({'page_num': i, 'raw_text': raw})
    # Optionally detect repeated headers/footers
    if detect_headers_footers:
        # build frequency map of exact lines across pages
        line_freq = {}
        for p in pages:
            lines = [ln.strip() for ln in (p['raw_text'] or "").splitlines() if ln.strip()]
            unique_lines = set(lines)
            for ln in unique_lines:
                line_freq[ln] = line_freq.get(ln, 0) + 1
        # find lines present in >50% pages (heuristic)
        threshold = max(2, int(0.5 * max(1, len(pages))))
        repeated_lines = {ln for ln, cnt in line_freq.items() if cnt >= threshold}
    else:
        repeated_lines = set()
    # clean each page accounting for repeated header/footer lines
    results = []
    for p in pages:
        raw = p['raw_text'] or ""
        # remove repeated lines
        if repeated_lines:
            lines = [ln for ln in raw.splitlines() if ln.strip() not in repeated_lines]
            raw = "\n".join(lines)
        clean = clean_page_text(raw)
        results.append({
            'doc_id': doc_id,
            'page_num': p['page_num'],
            'raw_text': raw,
            'clean_text': clean,
            'source_path': path
        })
    return results

In [ ]:
def ingest_pdfs_in_dir(dir_path: str, output_path: str = "extracted_docs.jsonl", detect_headers_footers=True):
    """
    Walk a directory, extract every .pdf, and store page-level JSON lines with metadata.
    output_path is a JSONL containing page-level records.
    """
    records = []
    for root, _, files in os.walk(dir_path):
        for fname in files:
            if fname.lower().endswith('.pdf'):
                fp = os.path.join(root, fname)
                print(f"Extracting: {fp}")
                pages = extract_text_from_pdf(fp, detect_headers_footers=detect_headers_footers)
                records.extend(pages)
    # Save to JSONL
    with open(output_path, 'w', encoding='utf-8') as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")
    print(f"Saved {len(records)} page records to {output_path}")
    return output_path

In [ ]:
from google.colab import files
uploaded = files.upload()  # a dialog to upload files

# Save uploaded files to /content/pdfs
os.makedirs("/content/pdfs", exist_ok=True)
for fname, content in uploaded.items():
    with open(f"/content/pdfs/{fname}", "wb") as f:
        f.write(content)
# then run ingestion on that folder
ingest_pdfs_in_dir("/content/pdfs", output_path="/content/extracted_docs.jsonl")


Saving toaz.info-process-control-instrumentation-technology-8th-ed-pr_d567a3ff3f917eab73b02a5426cce99d.pdf to toaz.info-process-control-instrumentation-technology-8th-ed-pr_d567a3ff3f917eab73b02a5426cce99d.pdf
Extracting: /content/pdfs/toaz.info-process-control-instrumentation-technology-8th-ed-pr_d567a3ff3f917eab73b02a5426cce99d.pdf


Saved 685 page records to /content/extracted_docs.jsonl


'/content/extracted_docs.jsonl'

In [ ]:
import os

folder_path = "/content/drive/MyDrive/rag_pdfs"
os.makedirs(folder_path, exist_ok=True)

print("Folder created:", folder_path)


Folder created: /content/drive/MyDrive/rag_pdfs


In [ ]:
for fname, data in uploaded.items():
    with open(f"/content/drive/MyDrive/rag_pdfs/{fname}", "wb") as f:
        f.write(data)

In [ ]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# put your PDFs into a folder: /content/drive/MyDrive/rag_pdfs/
ingest_pdfs_in_dir("/content/drive/MyDrive/rag_pdfs", output_path="/content/drive/MyDrive/extracted_docs.jsonl")


Extracting: /content/drive/MyDrive/rag_pdfs/toaz.info-process-control-instrumentation-technology-8th-ed-pr_d567a3ff3f917eab73b02a5426cce99d.pdf


Saved 685 page records to /content/drive/MyDrive/extracted_docs.jsonl


'/content/drive/MyDrive/extracted_docs.jsonl'

In [ ]:
file_path = "/content/drive/MyDrive/extracted_docs.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    for i in range(5):        # change 5 → 20 if you want more
        print(f.readline())


{"doc_id": "toaz.info-process-control-instrumentation-technology-8th-ed-pr_d567a3ff3f917eab73b02a5426cce99d.pdf", "page_num": 1, "raw_text": "Process Control Instrumentation\nTechnology\nCurtis D. Johnson\nEighth Edition\nISBN 978-1-29202-601-5\n9 781292 026015\nProcess\nControl\nInstrumentation\nTechnology\nJohnson\n8e", "clean_text": "Process Control Instrumentation\nTechnology\nCurtis D. Johnson\nEighth Edition\nISBN 978-1-29202-601-\n9 781292 026015\nProcess\nControl\nInstrumentation\nTechnology\nJohnson\n8e", "source_path": "/content/drive/MyDrive/rag_pdfs/toaz.info-process-control-instrumentation-technology-8th-ed-pr_d567a3ff3f917eab73b02a5426cce99d.pdf"}

{"doc_id": "toaz.info-process-control-instrumentation-technology-8th-ed-pr_d567a3ff3f917eab73b02a5426cce99d.pdf", "page_num": 2, "raw_text": "Process Control Instrumentation\nTechnology\nCurtis D. Johnson\nEighth Edition", "clean_text": "Process Control Instrumentation\nTechnology\nCurtis D. Johnson\nEighth Edition", "source_pa

In [ ]:
!pip install transformers

In [ ]:
import json
from transformers import AutoTokenizer
import math

# Tokenizer (use same as your LLM family)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
# Note: We use GPT2 tokenizer because it's fast & widely used for chunking.

def count_tokens(text):
    return len(tokenizer.encode(text))

def chunk_text(text, chunk_size=300, overlap=50):
    tokens = tokenizer.encode(text)
    chunks = []

    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens)

        chunks.append(chunk_text)
        start += (chunk_size - overlap)  # step forward with overlap

    return chunks

def create_chunks_from_jsonl(jsonl_path, output_path="chunks.jsonl"):
    chunks = []
    chunk_id = 0

    with open(jsonl_path, "r", encoding="utf-8") as f:
        pages = [json.loads(line) for line in f]

    current_doc = None
    current_text = ""
    current_start_page = None

    for p in pages:
        doc_id = p["doc_id"]
        clean_text = p["clean_text"]
        page_num = p["page_num"]

        # New document starts
        if current_doc is None:
            current_doc = doc_id
            current_start_page = page_num

        # If doc_id changes → process previous doc
        if doc_id != current_doc:
            text_chunks = chunk_text(current_text)

            for c in text_chunks:
                chunks.append({
                    "doc_id": current_doc,
                    "chunk_id": chunk_id,
                    "page_start": current_start_page,
                    "page_end": previous_page,
                    "text": c
                })
                chunk_id += 1

            # Reset
            current_doc = doc_id
            current_text = ""
            current_start_page = page_num

        # Accumulate text
        current_text += " " + clean_text
        previous_page = page_num

    # After loop: process final doc
    text_chunks = chunk_text(current_text)
    for c in text_chunks:
        chunks.append({
            "doc_id": current_doc,
            "chunk_id": chunk_id,
            "page_start": current_start_page,
            "page_end": previous_page,
            "text": c
        })
        chunk_id += 1

    # Save chunks.jsonl
    with open(output_path, "w", encoding="utf-8") as f:
        for c in chunks:
            f.write(json.dumps(c) + "\n")

    print(f"Saved {len(chunks)} chunks to {output_path}")
    return output_path

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
create_chunks_from_jsonl(
    "/content/drive/MyDrive/extracted_docs.jsonl",
    "/content/drive/MyDrive/chunks.jsonl"
)

Token indices sequence length is longer than the specified maximum sequence length for this model (302027 > 1024). Running this sequence through the model will result in indexing errors


Saved 1209 chunks to /content/drive/MyDrive/chunks.jsonl


'/content/drive/MyDrive/chunks.jsonl'

In [ ]:
#------Embeddings--------
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-small-en")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#-------Storing Vector databases-------

import json
import chromadb
from chromadb.config import Settings

# Configuration
chroma_dir = "/content/drive/MyDrive/rag_vector_db"
chunks_path = "/content/drive/MyDrive/chunks.jsonl"
batch_size = 256

# Initialize Chroma client
client = chromadb.PersistentClient(path=chroma_dir)
collection = client.get_or_create_collection(name="rag_collection")

# Load chunks
chunks = []
with open(chunks_path, "r", encoding="utf-8") as f:
    for line in f:
        chunks.append(json.loads(line))

print(f"Total chunks: {len(chunks)}")

# Prepare data
ids = [str(c["chunk_id"]) for c in chunks]
texts = [c["text"] for c in chunks]
metadatas = [
    {
        "doc_id": c["doc_id"],
        "page_start": c["page_start"],
        "page_end": c["page_end"]
    }
    for c in chunks
]

# Compute embeddings
# Ensure model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = model.encode(texts, show_progress_bar=True).tolist()

# Add to collection in batches
for i in range(0, len(ids), batch_size):
    j = min(i + batch_size, len(ids))
    collection.add(
        ids=ids[i:j],
        documents=texts[i:j],
        metadatas=metadatas[i:j],
        embeddings=embeddings[i:j]
    )
    print(f"Added batch {i}-{j}")

print("Chroma DB population complete.")

Total chunks: 1209


Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Added batch 0-256
Added batch 256-512
Added batch 512-768
Added batch 768-1024
Added batch 1024-1209
Chroma DB population complete.


In [ ]:
# ====== Robust re-add embeddings to Chroma collection (run in Colab) ======
# Pre-req: mount drive and install/import chromadb & sentence-transformers, and load embed_model
# Example:
# from sentence_transformers import SentenceTransformer
# embed_model = SentenceTransformer("BAAI/bge-small-en")

import json, os, math, traceback
import chromadb
from chromadb.config import Settings

# Paths
chroma_dir = "/content/drive/MyDrive/rag_vector_db"
chunks_path = "/content/drive/MyDrive/chunks.jsonl"
collection_name = "rag_collection"

# Create client safely (handle versions)
try:
    client = chromadb.Client(Settings(persist_directory=chroma_dir))
except Exception as e:
    print("Settings(...) failed, falling back to default client:", type(e).__name__)
    client = chromadb.Client()

# Get or create collection
try:
    collection = client.get_or_create_collection(name=collection_name)
except Exception:
    # older/newer versions might accept different signatures
    collection = client.get_or_create_collection(collection_name)

# Quick check: list files in chroma_dir to sanity-check persistence
print("Contents of chroma_dir:", os.listdir(chroma_dir) if os.path.exists(chroma_dir) else "NOT FOUND")

# Load chunks from disk
chunks = []
with open(chunks_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            chunks.append(json.loads(line))
        except Exception as e:
            print("Skipping bad line:", e)
print("Loaded chunks from disk:", len(chunks))

if len(chunks) == 0:
    raise RuntimeError("No chunks found in chunks.jsonl — aborting. Check the file path.")

# Prepare data lists
ids = [str(c["chunk_id"]) for c in chunks]
docs = [c["text"] for c in chunks]
metadatas = [{"doc_id": c.get("doc_id"), "page_start": c.get("page_start"), "page_end": c.get("page_end")} for c in chunks]

# OPTIONAL: If you already computed embeddings earlier and saved them separately, load them here.
# Otherwise compute now.
print("Computing embeddings for all chunks (this may take a while)...")
try:
    embeddings = embed_model.encode(docs, show_progress_bar=True)
    # ensure list of lists (json serializable)
    embeddings = [e.tolist() if hasattr(e, "tolist") else e for e in embeddings]
except Exception as e:
    print("Failed to compute embeddings:", type(e).__name__)
    traceback.print_exc()
    raise

print("Embeddings shape:", len(embeddings), "x", len(embeddings[0]) if len(embeddings)>0 else 0)

# Add to collection in batches to avoid memory pressure / timeouts
batch_size = 256
total = len(ids)
for start in range(0, total, batch_size):
    end = min(start + batch_size, total)
    try:
        collection.add(
            ids=ids[start:end],
            documents=docs[start:end],
            metadatas=metadatas[start:end],
            embeddings=embeddings[start:end]
        )
        print(f"Added batch {start}:{end}")
    except Exception as e:
        print("Failed to add batch", start, end, type(e).__name__)
        traceback.print_exc()
        # try smaller sub-batches
        for i in range(start, end, 64):
            j = min(i+64, end)
            try:
                collection.add(
                    ids=ids[i:j],
                    documents=docs[i:j],
                    metadatas=metadatas[i:j],
                    embeddings=embeddings[i:j]
                )
                print(f"  Added sub-batch {i}:{j}")
            except Exception as e2:
                print("  Sub-batch failed:", i, j, type(e2).__name__)
                traceback.print_exc()

# Persist (best-effort)
try:
    client.persist()
    print("client.persist() called.")
except Exception as e:
    print("client.persist() not supported or failed:", type(e).__name__, e)

# Final verification
try:
    cnt = collection.count()
    print("Final collection.count() ->", cnt)
except Exception:
    # fallback: try inspecting collection.get()
    try:
        info = collection.get()
        ids_in = info.get("ids", [])
        print("Final ids count (via get):", len(ids_in))
    except Exception as e:
        print("Could not verify collection size:", type(e).__name__, e)

print("Done. If count still 0, check for multiple chroma directories or re-run with upgraded chromadb.")


Contents of chroma_dir: ['chroma.sqlite3', '4068019f-72ba-44dd-a650-c43318204d06']
Loaded chunks from disk: 1209
Computing embeddings for all chunks (this may take a while)...


Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Embeddings shape: 1209 x 384
Added batch 0:256
Added batch 256:512
Added batch 512:768
Added batch 768:1024
Added batch 1024:1209
client.persist() not supported or failed: AttributeError 'Client' object has no attribute 'persist'
Final collection.count() -> 1209
Done. If count still 0, check for multiple chroma directories or re-run with upgraded chromadb.


In [ ]:
#---- Loading croma collection------

import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

# Load embedding model
embed_model = SentenceTransformer("BAAI/bge-small-en")

# Load Chroma DB
chroma_dir = "/content/drive/MyDrive/rag_vector_db"
client = chromadb.Client(Settings(persist_directory=chroma_dir))

collection = client.get_or_create_collection("rag_collection")

print("Chroma loaded successfully!")


Chroma loaded successfully!


In [ ]:
#----Load a small LLM----

from transformers import pipeline

rag_llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256,
    device_map="auto"
)

Device set to use cuda:0


In [ ]:
#------Retrieval Function------

def retrieve_relevant_chunks(query, n_results=4):
    # Embed user query
    query_emb = embed_model.encode([query]).tolist()

    # Query Chroma
    results = collection.query(
        query_embeddings=query_emb,
        n_results=n_results
    )

    retrieved = []
    for text, meta in zip(results["documents"][0], results["metadatas"][0]):
        retrieved.append({
            "text": text,
            "page_start": meta["page_start"],
            "page_end": meta["page_end"]
        })
    return retrieved

In [ ]:
#-----Build the RAG Prompt-----

def build_prompt(query, retrieved_chunks):
    context = "\n\n".join(
        [f"(Pages {c['page_start']}-{c['page_end']}): {c['text']}" for c in retrieved_chunks]
    )

    prompt = f"""
You are a helpful assistant. Answer the question strictly using the context below.
If the answer is not present in the context, say: "The answer is not available in the document."

### Context:
{context}

### Question:
{query}

### Answer:
"""
    return prompt


In [ ]:
def answer_with_rag(query):
    chunks = retrieve_relevant_chunks(query)
    if not chunks:
        return "No relevant chunks retrieved.", []

    prompt = build_prompt(query, chunks)
    response = rag_llm(prompt)[0]["generated_text"]
    return response, chunks


In [ ]:
response, sources = answer_with_rag("Explain the working principle of instrumentation amplifier")

print("ANSWER:\n", response)
print("\nSOURCES USED:\n", sources)


Token indices sequence length is longer than the specified maximum sequence length for this model (1312 > 512). Running this sequence through the model will result in indexing errors


ANSWER:
 The transfer function of this amplifier is given by 2R R V = a1 + 1ba 3b(V - V) (41) out R R 2 G The input impedance is very high, and the output impedance very low. Many IC manufacturers provide this circuit with fixed differential gain andR, but allow the user to insert external R so the desired gain can be selected. They can thus ensure a high CMR.

SOURCES USED:
 [{'text': '. The most common circuit for this amplifier is shown in Figure34. Notice that the\ncircuit uses two pairs of matched resistors, R andR . When the matching is perfect and the\n1\nop amp is ideal, the transfer function for this amplifier is given by (see Appendix 6)\nR\nV = 2 (V - V ) (40)\nout R 2\nIf the resistors are not well matched, the CMR will be poor. The circuit of Figure 33has\na disadvantage in that its input impedance is not very high and, further, is not the same for\nthe two inputs. For this reason, voltage followers are often used on the input to provide high\ninput impedance. The result i